# Clustering Crypto

In [87]:
# Initial imports
import pandas as pd
import hvplot.pandas

import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [88]:
# Load the crypto_data.csv dataset.
crypto_df=pd.read_csv("crypto_data.csv", index_col=0)


In [28]:
# Keep all the cryptocurrencies that are being traded.

crypto_df.head(10)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [29]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df[crypto_df.Algorithm.isna() == False]
print(crypto_df.shape)
crypto_df.head(10)

(1252, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [30]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns="IsTrading")
print(crypto_df.shape)
crypto_df.head(10)

(1252, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [31]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df[~(crypto_df.isna() == True).any(axis=1)]
print(crypto_df.shape)
crypto_df.head(10)

(744, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [89]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[~(crypto_df.TotalCoinsMind <= 0)]
print(crypto_df.shape)
crypto_df.head(10)

AttributeError: 'DataFrame' object has no attribute 'TotalCoinsMind'

In [34]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = pd.DataFrame(crypto_df.CoinName)
print(crypto_name_df.shape)
crypto_name_df.head()

(744, 1)


,CoinName
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [35]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns="CoinName")
print(crypto_df.shape)
crypto_df.head(10)

(744, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000


In [36]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head(10)

(744, 113)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
crypto_scaled = scaler.fit_transform(X)
crypto_scaled[0:5]

array([[-0.10047714, -0.03668886, -0.03668644, -0.03668644, -0.03668644,
        -0.03668644, -0.05191741, -0.09016696, -0.03668644, -0.05191741,
        -0.05191741, -0.03668644, -0.03668644, -0.18257419, -0.05191741,
        -0.03668644, -0.03668644, -0.08225509, -0.03668644, -0.0974575 ,
        -0.06362848, -0.03668644, -0.03668644, -0.16620562, -0.03668644,
        -0.03668644, -0.13848495, -0.03668644, -0.03668644, -0.07352146,
        -0.05191741, -0.03668644, -0.03668644, -0.03668644, -0.06362848,
        -0.03668644, -0.07352146, -0.09016696, -0.09016696, -0.03668644,
        -0.03668644, -0.12250233, -0.12803688, -0.15291752, -0.03668644,
        -0.08225509, -0.03668644, -0.03668644, -0.06362848, -0.16188544,
        -0.03668644, -0.03668644, -0.03668644, -0.07352146, -0.1786061 ,
        -0.32732684, -0.03668644, -0.0974575 , -0.08225509, -0.05191741,
        -0.03668644,  1.42714214, -0.06362848, -0.03668644, -0.03668644,
        -0.03668644, -0.08225509, -0.06362848, -0.0

### Deliverable 2: Reducing Data Dimensions Using PCA

In [38]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.17390317,  1.2719689 , -0.42979493],
       [-0.1589508 ,  1.2713572 , -0.43036593],
       [-0.16294998,  0.77951004, -0.31837575],
       ...,
       [-0.2910146 , -2.01553334,  0.29948229],
       [-0.08139512,  0.87650692, -0.15380354],
       [-0.17359863,  1.27195644, -0.42980656]])

In [39]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df)
print(pcs_df.shape)
pcs_df.head(10)

(744, 3)


,PC 1,PC 2,PC 3
"(Scrypt, PoW/PoS, 41.99995383, 42)",-0.173903,1.271969,-0.429795
"(Scrypt, PoW/PoS, 1055184902.04, 532000000)",-0.158951,1.271357,-0.430366
"(SHA-256, PoW/PoS, 0.0, 0)",-0.162950,0.779510,-0.318376
"(X13, PoW/PoS, 29279424622.5027, 314159265359)",0.304997,2.016034,-0.493353
"(SHA-256, PoW, 17927175.0, 21000000)",-0.230315,-1.494986,0.053560
"(Ethash, PoW, 107684222.6865, 0)",-0.310352,-2.157877,0.186167
"(Scrypt, PoW, 63039243.3000049, 84000000)",-0.240629,-1.002553,-0.057884
"(X11, PoW/PoS, 9031294.37563393, 22000000)",-0.241769,1.454396,-0.355669
"(CryptoNight-V7, PoW, 17201143.1449131, 0)",-0.294596,-2.340106,0.169402
"(Ethash, PoW, 113359703.0, 210000000)",-0.310271,-2.157881,0.186164


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [40]:
# Create an elbow curve to find the best value for K.

# Initialize empty inertia list
inertia = []
# Initialize k range
k = list(range(1,11))
# looping through k list
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Define a DataFrame to plot the Elbow Curve using hvplot
elbow_data = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


C:\Users\Abdulahmeed Bello\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\Abdulahmeed Bello\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\Abdulahmeed Bello\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\Abdulahmeed Bello\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning:

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [41]:
# Initialize the K-Means model.
km_model = KMeans(n_clusters=4, random_state=0)

# Fit the model
km_model.fit(pcs_df)

# Predict clusters
predictions = km_model.predict(pcs_df)
predictions

C:\Users\Abdulahmeed Bello\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


array([0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,

In [54]:
import pandas as pd
import plotly.express as px
import hvplot.pandas
from sklearn.cluster import KMeans

In [56]:
# Initializing model with K = 3 (since we already know there are three classes of clusters plants)
model = KMeans(n_clusters=3, random_state=5)
model

KMeans(n_clusters=3, random_state=5)

In [74]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat('crypto_df', 'pcs_df', 'axis=1', join='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_name_df.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = km_model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

C:\Users\Abdulahmeed Bello\AppData\Local\Temp\ipykernel_26084\2892801235.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  clustered_df = pd.concat('crypto_df', 'pcs_df', 'axis=1', join='inner')


TypeError: concat() got multiple values for argument 'join'

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [79]:
crypto_df=pd.read_csv("crypto_data.csv")

In [90]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="petal_width", y="sepal_length", z="petal_length", color="class", symbol="class", size="sepal_width",width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


NameError: name 'clustered_df' is not defined

In [21]:
# Create a table with tradable cryptocurrencies.
tradable_crypto_table = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm',
                                                  'TotalCoinsMined', 'Class'],)
tradable_crypto_table

NameError: name 'clustered_df' is not defined

In [22]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df)} tradable cryptocurrencies.')

NameError: name 'clustered_df' is not defined

In [75]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

NameError: name 'clustered_df' is not defined

In [76]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_cluster_scaled, columns=['TotalCoinSupply', 'TotalCoin'])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df.CoinName

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df.Class

plot_df.head(10)

NameError: name 'X_cluster_scaled' is not defined

In [91]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(X='TotalCoinsMined', y='TotalCoinsupply', hover_cols='Coin')


NameError: name 'plot_df' is not defined